# Software Requirements Specification for Chemistry Code
## Samuel J. Crawford




# Table of Contents
<a id="Sec:ToC"></a>

An outline of all sections included in this SRS is recorded here for easy reference.

- [Table of Contents](#Sec:ToC)
- [Reference Material](#Sec:RefMat)
 - [Table of Symbols](#Sec:ToS)
 - [Abbreviations and Acronyms](#Sec:TAbbAcc)
- [Introduction](#Sec:Intro)
 - [Purpose of Document](#Sec:DocPurpose)
 - [Scope of Requirements](#Sec:ReqsScope)
 - [Organization of Document](#Sec:DocOrg)
- [Specific System Description](#Sec:SpecSystDesc)
- [Solution Characteristics Specification](#Sec:SolCharSpec)
 - [Theoretical Models](#Sec:TMs)
 - [Data Definitions](#Sec:DDs)
- [Requirements](#Sec:Requirements)
 - [Functional Requirements](#Sec:FRs)
 - [Non-Functional Requirements](#Sec:NFRs)
- [References](#Sec:References)

# Reference Material
<a id="Sec:RefMat"></a>

This section records information for easy reference.

## Table of Symbols
<a id="Sec:ToS"></a>

The symbols used in this document are summarized in the [Table of Symbols](#Table:ToS) along with their units. The symbols are listed in alphabetical order.

<a id="Table:ToS"></a>

|Symbol|Description|Units|
|:--- |:--- |:--- |
|$0$|Zero vector|--|
|$A$|Generic matrix|--|
|$b$|Generic vector|--|
|$C$|Compound data type|--|
|$c$|Generic vector|--|
|$E$|Element data type|--|
|$r$|Representation of a chemical equation|--|
|$x$|Generic vector|--|


## Abbreviations and Acronyms
<a id="Sec:TAbbAcc"></a>

<a id="Table:TAbbAcc"></a>

|Abbreviation|Full Form|
|:--- |:--- |
|ChemCode|Chemistry Code|
|DD|Data Definition|
|SRS|Software Requirements Specification|
|TM|Theoretical Model|


# Introduction
<a id="Sec:Intro"></a>

Chemical equations are common ways of representing chemical reactions but they must be balanced <a href=#lund2023>lund2023</a> . This process of balancing a chemical equation involves introducing coefficients before each chemical formula such that there are the same number of atoms of each element on the reactant and product sides of the chemical equation. Because balancing must be done before a given chemical reaction can be used <a href=#lund2023>lund2023</a> , it is useful to have a tool to automatically do this. This would improve the productivity of scientists and engineers and reduce the potential for human error. This program should balance a given chemical equation if it is feasible and if it is not, it should provide a descriptive message communicating this to the user. The program that performs these tasks as documented here will be called Chemistry Code (ChemCode).

The following section provides an overview of the Software Requirements Specification (SRS) for ChemCode. This section explains the purpose of this document, the scope of the requirements, the characteristics of the intended reader, and the organization of the document.

## Purpose of Document
<a id="Sec:DocPurpose"></a>

The primary purpose of this document is to record the requirements of the Chemistry Code. Goals, assumptions, theoretical models, definitions, and other model derivation information are specified, allowing the reader to fully understand and verify the purpose and scientific basis of ChemCode. With the exception of [system constraints](#Sec:SysConstraints), this SRS will remain abstract, describing what problem is being solved, but not how to solve it.

This document will be used as a starting point for subsequent development phases, including writing the design specification and the software verification and validation plan. The design document will show how the requirements are to be realized, including decisions on the numerical algorithms and programming environment. The verification and validation plan will show the steps that will be used to increase confidence in the software documentation and the implementation. Although the SRS fits in a series of documents that follow the so-called waterfall model, the actual development process is not constrained in any way. Even when the waterfall model is not followed, as Parnas and Clements point out <a href=#parnasClements1986>parnasClements1986</a> , the most logical way to present the documentation is still to "fake" a rational design process.

## Scope of Requirements
<a id="Sec:ReqsScope"></a>

The scope of the requirements includes all chemical equations with at most one more compound than element. Furthermore, it also includes all inputted chemical formulas that describe real chemical compounds, are formatted following a set of conventions, and only consist of atomic symbols and subscripts.

## Organization of Document
<a id="Sec:DocOrg"></a>

The organization of this document follows the template for an SRS for scientific computing software proposed by <a href=#koothoor2013>koothoor2013</a>  and <a href=#smithLai2005>smithLai2005</a> . The presentation follows the standard pattern of presenting goals, theories, definitions, and assumptions. For readers that would like a more bottom up approach, they can start reading the [instance models](#Sec:IMs) and trace back to find any additional information they require.

The [goal statements](#Sec:GoalStmt) are refined to the theoretical models and the [theoretical models](#Sec:TMs) to the [instance models](#Sec:IMs).

# Specific System Description
<a id="Sec:SpecSystDesc"></a>

This section first presents the problem description, which gives a high-level view of the problem to be solved. This is followed by the solution characteristics specification, which presents the assumptions, theories, and definitions that are used.

## Solution Characteristics Specification
<a id="Sec:SolCharSpec"></a>

The instance models that govern ChemCode are presented in the [Instance Model Section](#Sec:IMs). The information to understand the meaning of the instance models and their derivation is also presented, so that the instance models can be verified.

### Theoretical Models
<a id="Sec:TMs"></a>

This section focuses on the general equations and laws that ChemCode is based on.
<br>
<a id="TM:intLinProg"></a>
<table class=\"tdefn\">
<tr>
<th>Refname</th>
<td>
<b>TM:intLinProg</b>
</td>
</tr>
<tr>
<th>Label</th>
<td>

Integer linear program
</td>
</tr>
<tr>
<th>Equation</th>
<td>
$$\begin{cases}\symbf{c} \symbf{x}, & 1\\\symbf{A} \symbf{x}\leq{}\symbf{b}, & 2\\\symbf{x}\geq{}\symbf{0}, & 3\end{cases}$$
</td>
</tr>
<tr>
<th>Description</th>
<td>

<li>$c$ is the generic vector (Unitless)</li>
<li>$x$ is the generic vector (Unitless)</li>
<li>$A$ is the generic matrix (Unitless)</li>
<li>$b$ is the generic vector (Unitless)</li>
<li>$0$ is the zero vector (Unitless)</li>
</td>
</tr>
<tr>
<th>Notes</th>
<td>

The above equation gives the canonical form of an integer linear program, which is "a mathematical optimization or feasibility program in which some or all of the variables are restricted to be integers [and] the objective function and the constraints (other than the integer constraints) are linear" <a href=#ilpWiki>ilpWiki</a> 

The values of $x$ are unknown and will be solved for
</td>
</tr>
<tr>
<th>Source</th>
<td>

<a href=#ilpWiki>ilpWiki</a> 
</td>
</tr>
<tr>
<th>RefBy</th>
<td>


</td>
</tr>
</table>

### Data Definitions
<a id="Sec:DDs"></a>

This section collects and defines all the data needed to build the instance models.
<br>
<a id="DD:elementType"></a>
<table class=\"ddefn\">
<tr>
<th>Refname</th>
<td>
<b>DD:elementType</b>
</td>
</tr>
<tr>
<th>Label</th>
<td>

Element data type
</td>
</tr>
<tr>
<th>Symbol</th>
<td>

$E$
</td>
</tr>
<tr>
<th>Equation</th>
<td>
$$E=\{\text{H},\text{He},\text{Li},\text{Be},\text{B},\text{C},\text{N},\text{O},\text{F},\text{Ne},\text{Na},\text{Mg},\text{Al},\text{Si},\text{P},\text{S},\text{Cl},\text{Ar},\text{K},\text{Ca},\text{Sc},\text{Ti},\text{V},\text{Cr},\text{Mn},\text{Fe},\text{Co},\text{Ni},\text{Cu},\text{Zn},\text{Ga},\text{Ge},\text{As},\text{Se},\text{Br},\text{Kr},\text{Rb},\text{Sr},\text{Y},\text{Zr},\text{Nb},\text{Mo},\text{Tc},\text{Ru},\text{Rh},\text{Pd},\text{Ag},\text{Cd},\text{In},\text{Sn},\text{Sb},\text{Te},\text{I},\text{Xe},\text{Cs},\text{Ba},\text{La},\text{Ce},\text{Pr},\text{Nd},\text{Pm},\text{Sm},\text{Eu},\text{Gd},\text{Tb},\text{Dy},\text{Ho},\text{Er},\text{Tm},\text{Yb},\text{Lu},\text{Hf},\text{Ta},\text{W},\text{Re},\text{Os},\text{Ir},\text{Pt},\text{Au},\text{Hg},\text{Tl},\text{Pb},\text{Bi},\text{Po},\text{At},\text{Rn},\text{Fr},\text{Ra},\text{Ac},\text{Th},\text{Pa},\text{U},\text{Np},\text{Pu},\text{Am},\text{Cm},\text{Bk},\text{Cf},\text{Es},\text{Fm},\text{Md},\text{No},\text{Lr},\text{Rf},\text{Db},\text{Sg},\text{Bh},\text{Hs},\text{Mt},\text{Ds},\text{Rg},\text{Cn},\text{Nh},\text{Fl},\text{Mc},\text{Lv},\text{Ts},\text{Og}\}$$
</td>
</tr>
<tr>
<th>Description</th>
<td>

<li>$E$ is the element data type (Unitless)</li>
</td>
</tr>
<tr>
<th>Notes</th>
<td>

A type representing each element from <a href=#elemListWiki>elemListWiki</a> 
</td>
</tr>
<tr>
<th>Source</th>
<td>

<a href=#smithChemSpec>smithChemSpec</a> 
</td>
</tr>
<tr>
<th>RefBy</th>
<td>


</td>
</tr>
</table>
<br>
<a id="DD:compoundType"></a>
<table class=\"ddefn\">
<tr>
<th>Refname</th>
<td>
<b>DD:compoundType</b>
</td>
</tr>
<tr>
<th>Label</th>
<td>

Compound data type
</td>
</tr>
<tr>
<th>Symbol</th>
<td>

$C$
</td>
</tr>
<tr>
<th>Equation</th>
<td>
$$C=\text{sequence of}\,\text{tuple of}\,\left(\text{elem}\in{}\{\text{H},\text{He},\text{Li},\text{Be},\text{B},\text{C},\text{N},\text{O},\text{F},\text{Ne},\text{Na},\text{Mg},\text{Al},\text{Si},\text{P},\text{S},\text{Cl},\text{Ar},\text{K},\text{Ca},\text{Sc},\text{Ti},\text{V},\text{Cr},\text{Mn},\text{Fe},\text{Co},\text{Ni},\text{Cu},\text{Zn},\text{Ga},\text{Ge},\text{As},\text{Se},\text{Br},\text{Kr},\text{Rb},\text{Sr},\text{Y},\text{Zr},\text{Nb},\text{Mo},\text{Tc},\text{Ru},\text{Rh},\text{Pd},\text{Ag},\text{Cd},\text{In},\text{Sn},\text{Sb},\text{Te},\text{I},\text{Xe},\text{Cs},\text{Ba},\text{La},\text{Ce},\text{Pr},\text{Nd},\text{Pm},\text{Sm},\text{Eu},\text{Gd},\text{Tb},\text{Dy},\text{Ho},\text{Er},\text{Tm},\text{Yb},\text{Lu},\text{Hf},\text{Ta},\text{W},\text{Re},\text{Os},\text{Ir},\text{Pt},\text{Au},\text{Hg},\text{Tl},\text{Pb},\text{Bi},\text{Po},\text{At},\text{Rn},\text{Fr},\text{Ra},\text{Ac},\text{Th},\text{Pa},\text{U},\text{Np},\text{Pu},\text{Am},\text{Cm},\text{Bk},\text{Cf},\text{Es},\text{Fm},\text{Md},\text{No},\text{Lr},\text{Rf},\text{Db},\text{Sg},\text{Bh},\text{Hs},\text{Mt},\text{Ds},\text{Rg},\text{Cn},\text{Nh},\text{Fl},\text{Mc},\text{Lv},\text{Ts},\text{Og}\},\text{count}\in{}\mathbb{R}\right)$$
</td>
</tr>
<tr>
<th>Description</th>
<td>

<li>$C$ is the compound data type (Unitless)</li>
</td>
</tr>
<tr>
<th>Notes</th>
<td>

A type representing a compound
</td>
</tr>
<tr>
<th>Source</th>
<td>

--
</td>
</tr>
<tr>
<th>RefBy</th>
<td>


</td>
</tr>
</table>

# Requirements
<a id="Sec:Requirements"></a>

This section provides the functional requirements, the tasks and behaviours that the software is expected to complete, and the non-functional requirements, the qualities that the software is expected to exhibit.

## Functional Requirements
<a id="Sec:FRs"></a>

This section provides the functional requirements, the tasks and behaviours that the software is expected to complete.

<div id=\"inputValues\">
Input-Values: Input the values from [Tab:ReqInputs](#Table:ReqInputs).

</div>
<div id=\"convertMatrix\">
Convert-to-Matrix: Convert the inputted chemical equation to matrix form.

</div>
<div id=\"feasOut\">
Feasible-Output: If the inputted chemical equation is feasible, output its balanced form with the smallest positive integer coefficients possible.

</div>
<div id=\"infeasOut\">
Infeasible-Output: If the inputted chemical equation is infeasible, output a descriptive message.

</div>

<a id="Table:ReqInputs"></a>

|Symbol|Description|Units|
|:--- |:--- |:--- |
|$r$|Representation of a chemical equation|--|


## Non-Functional Requirements
<a id="Sec:NFRs"></a>

This section provides the non-functional requirements, the qualities that the software is expected to exhibit.

<div id=\"accurate\">
Accurate: Chemical equations are only useful if they are balanced, so computed coefficients should be exact.

</div>
<div id=\"verifiable\">
Verifiable: The code is tested with complete verification and validation plan.

</div>
<div id=\"reusable\">
Reusable: The code is modularized.

</div>
<div id=\"portable\">
Portable: The code is able to be run in different environments.

</div>

# References
<a id="Sec:References"></a>
<a id="koothoor2013"></a>
[1]: Koothoor, Nirmitha. <em>A document drive approach to certifying scientific computing software</em>. McMaster University, Hamilton, ON, Canada: 2013. Print.
<a id="lund2023"></a>
[2]: Lund, Lance. <em>Introduction to Chemistry</em>. Cambridge and Coon Rapids, MN, USA: LibreTexts, January, 2023. <a href="https://chem.libretexts.org/Courses/Anoka-Ramsey_Community_College/Introduction_to_Chemistry">https://chem.libretexts.org/Courses/Anoka-Ramsey_Community_College/Introduction_to_Chemistry</a>. Print.
<a id="parnasClements1986"></a>
[3]: Parnas, David L. and Clements, P. C. "A rational design process: How and why to fake it." <em>IEEE Transactions on Software Engineering</em>, vol. 12, no. 2, Washington, USA: February, 1986. pp. 251&ndash;257. Print.
<a id="smithChemSpec"></a>
[4]: Smith, W. Spencer. <em>Assignment 2</em>. February, 2020. <a href="https://gitlab.cas.mcmaster.ca/smiths/se2aa4_cs2me3/-/blob/master/Assignments/PreviousYears/2020/A2-ChemReacts/A2.pdf">https://gitlab.cas.mcmaster.ca/smiths/se2aa4_cs2me3/-/blob/master/Assignments/PreviousYears/2020/A2-ChemReacts/A2.pdf</a>.
<a id="smithLai2005"></a>
[5]: Smith, W. Spencer and Lai, Lei. "A new requirements template for scientific computing." <em>Proceedings of the First International Workshop on Situational Requirements Engineering Processes - Methods, Techniques and Tools to Support Situation-Specific Requirements Engineering Processes, SREP'05</em>. Edited by PJ Agerfalk, N. Kraiem, and J. Ralyte, Paris, France: 2005. pp. 107&ndash;121. In conjunction with 13th IEEE International Requirements Engineering Conference,
<a id="ilpWiki"></a>
[6]: Wikipedia Contributors. <em>Integer programming</em>. March, 2023. <a href="https://en.wikipedia.org/wiki/Integer_programming">https://en.wikipedia.org/wiki/Integer_programming</a>.
<a id="elemListWiki"></a>
[7]: Wikipedia Contributors. <em>List of chemical elements</em>. January, 2023. <a href="https://en.wikipedia.org/wiki/List_of_chemical_elements">https://en.wikipedia.org/wiki/List_of_chemical_elements</a>.

